In [3]:
import os
import csv
import pandas as pd
from string import punctuation
from collections import Counter
from nltk import *
from nltk.corpus import wordnet as wn

In [4]:
count = 1
for filenames in os.walk('../data/wikipedia_txt'):
    for filename in filenames[2]:
        text = open('../data/wikipedia_txt/'+filename, 'rU').read()
        text = text.replace('--', ' ')
        p_tag = pos_tag(word_tokenize(text.lower()))
        word_tag = [(w[0].strip(punctuation), w[1]) for w in p_tag if len(w[0].strip(punctuation))>0]

        bi_word = []

        for w1, w2 in zip(word_tag, word_tag[1:]):
            bi_word.append((w1, w2))
            
        bi_ext = []

        for word in bi_word:
            if word[0][1] in ['JJ', 'RB', 'VB', 'VBG', 'VBZ', 'VBD', 'VBN', 'NN', 'NNS', 'NNP'] and word[1][1] in ['NN', 'NNS', 'NNP']:
                if word[0][1] in ['JJ', 'RB']:
                    if len(wn.synsets(word[0][0],pos=wn.ADJ)) > 0 or len(wn.synsets(word[0][0],pos=wn.ADV)) > 0: word1 = word[0][0]
                    else: word1 = None
                elif word[0][1] in ['VB', 'VBG', 'VBZ', 'VBD', 'VBN']:
                    if word[0][1] == 'VB' and len(wn.synsets(word[0][0], pos=wn.VERB)) > 0:
                        word1 = word[0][0]
                    elif wn.morphy(word[0][0], wn.VERB) != None and len(wn.synsets(wn.morphy(word[0][0], wn.VERB), pos=wn.VERB)) > 0:
                        word1 = word[0][0]
                elif word[0][1] == 'NNS':
                    if wn.morphy(word[0][0], wn.NOUN) != None: word1 = word[0][0]
                    else: word1 = None
                else:
                    if len(wn.synsets(word[0][0], pos=wn.NOUN)) > 0: word1 = word[0][0]
                    else: word1 = None

                if word[1][1] in ['NN', 'NNS', 'NNP']:
                    if word[0][1] == 'NNS':
                        if wn.morphy(word[1][0], wn.NOUN) != None: word2 = word[1][0]
                        else: word2 = None
                    else:
                        if len(wn.synsets(word[1][0], pos=wn.NOUN)) > 0: word2 = word[1][0]
                        else: word2 = None
                if word1 != None and word2 != None: bi_ext.append(word1+' '+word2)

        bi_count = Counter(bi_ext).most_common()
        
        with open('bigram_count.csv', 'a+') as csv_file:
            writer = csv.writer(csv_file) 
            for w in bi_count:
                writer.writerow([filename[:-4], w[0], w[1]])
        
        if count % 30 == 0 or count == 2974:
            if count == 2974:
                count=3000
            print('*'*int(count/30)+'-'*(100-int(count/30))+' '+str(int(count/30))+'%', end='\r', flush=True)
        count += 1

/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: 'U' mode is deprecated


In [6]:
wc_file = pd.read_csv('bigram_count.csv', names = ['article', 'word', 'count'])
wc_file[:5]

,article,word,count
0,Fox_Television_Studios,production arm,3
1,Fox_Television_Studios,century fox,3
2,Fox_Television_Studios,television studios,2
3,Fox_Television_Studios,fox television,2
4,Fox_Television_Studios,twentieth century,2


In [8]:
labels_file = pd.read_csv('../data/comp5046-labels.csv', index_col='article', names = ['web','article','label'], usecols=['article','label'] )
labels_file[:5]

,label
article,
WWEG,Entertainment
Vitamin_String_Quartet,Entertainment
Vidyasagar_Evening_College,Education
University_of_Belgrade_Faculty_of_Philosophy,Education
Today's_Best_Hits,Entertainment


In [7]:
word_table = wc_file.pivot_table(values='count', index='article', columns='word', fill_value=0)
word_table[:5]

word,10 league,10 year,1000th song,100th anniversary,100th company,100th episode,100th field,100th game,100th id,100th infantry,...,zoo coordinates,zoo entertainment,zoo gardens,zoo rabbi,zoological gardens,zoology bengali,zoology botany,zulu tribes,zurich premiership,zurich switzerland
article,,,,,,,,,,,,,,,,,,,,,
102.1_Bay_Radio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107_Squadron_(Israel),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109th_Infantry_Division_(German_Empire),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Division_(German_Empire),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Parachute_Battalion_(United_Kingdom),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
len(word_table)

2969

In [10]:
data = pd.merge(labels_file, word_table, left_index = True, right_index = True)
data[:5]

,label,10 league,10 year,1000th song,100th anniversary,100th company,100th episode,100th field,100th game,100th id,...,zoo coordinates,zoo entertainment,zoo gardens,zoo rabbi,zoological gardens,zoology bengali,zoology botany,zulu tribes,zurich premiership,zurich switzerland
article,,,,,,,,,,,,,,,,,,,,,
102.1_Bay_Radio,Entertainment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107_Squadron_(Israel),Public administration,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109th_Infantry_Division_(German_Empire),Public administration,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Division_(German_Empire),Public administration,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Parachute_Battalion_(United_Kingdom),Public administration,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
data.to_csv('bigram_training_data.csv')